<a href="https://colab.research.google.com/github/RyanZR/ColabDock-Vina/blob/main/ColabDock_Vina_(SD).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🍊 ColabDock Vina** (Single Docking)
This project is created based on [Lab.06 / IBM3202 – Molecular Docking on AutoDock](https://colab.research.google.com/github/pb3lab/ibm3202/blob/master/tutorials/lab06_docking.ipynb#scrollTo=WutUwTFo2e4j). It provide more dynamic options for visulisation and docking. All credits goes to the author of the mentioned python notebook. 

---
---
# **Setting the Environment for Molecular Docking**

Before starting, we need to install all the necessary software and dependecies to perform molecular docking. 

+ py3Dmol (https://pypi.org/project/py3Dmol/)
+ biopython (https://biopython.org/)
+ condacolab (https://github.com/con)
+ OpenBabel (https://github.com/openbabel/openbabel)
+ MGLtools (https://ccsb.scripps.edu/mgltools/)
+ AutoDock Vina (https://vina.scripps.edu/)

In [1]:
# @title **Install dependencies and softwares**
# @markdown It will take a few minutes. It will **restart** after the installation. 

%%capture
# Delete sample data
!rm -r /content/sample_data > /dev/null 2>&1

# Install dependencies
!pip install py3Dmol > /dev/null 2>&1
!pip install kora > /dev/null 2>&1

# Setup AutoDock Vina
!wget -qnc https://vina.scripps.edu/wp-content/uploads/sites/55/2020/12/autodock_vina_1_1_2_linux_x86.tgz > /dev/null 2>&1
!tar xzvf autodock_vina_1_1_2_linux_x86.tgz > /dev/null 2>&1
!rm autodock_vina_1_1_2_linux_x86.tgz -f > /dev/null 2>&1

# Setup Conda
!pip install -q condacolab > /dev/null 2>&1
import condacolab
condacolab.install_miniconda()
!conda config --set notify_outdated_conda false > /dev/null 2>&1

# Install other packages
!conda install -c conda-forge -c bioconda mgltools biopython openbabel=2.4.1 zlib --yes > /dev/null 2>&1
!rm /content/condacolab_install.log > /dev/null 2>&1

## **Setting up softwares and modules**
This set up some important alias, modules and folders into Google Colab.

In [1]:
# @title **Import Python modules**
# @markdown This allow Python accessible to the neccessary modules.

# Import modules
import os
import sys
import shutil
import py3Dmol
import ipywidgets
import urllib.request
import kora.install.rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from ipywidgets import interact,fixed,IntSlider
from Bio.PDB import *
from pathlib import Path
from google.colab import drive,files


# Code to hide input
class Hide:
  def __enter__(self):
    self._original_stdout = sys.stdout
    sys.stdout = open(os.devnull, "w")
  
  def __exit__(self, exc_type, exc_val, exc_tb):
    sys.stdout.close()
    sys.stdout = self._original_stdout

colors = ["red","orange","yellow","lime","green","cyan","teal","blue","violet","purple","pink","gray","brown","white","black"] 

print("> Installation done")
print("> Import done")
print("> Environment ready for docking")

> Installation done
> Import done
> Environment ready for docking


In [ ]:
# @title **Import Google Drive**
# @markdown This allow data to be stored in Google Drive.

# Flush and mount GDrive
with Hide():
  drive.flush_and_unmount()
  drive.mount("/content/drive", force_remount=True)

print("> Mounted at /content/drive")

In [2]:
# @title **Setup alias for Vina**
# @markdown This create an alias for AutoDock Vina.

# Setup alias for Vina
%alias vina /content/autodock_vina_1_1_2_linux_x86/bin/vina
%alias vina_split /content/autodock_vina_1_1_2_linux_x86/bin/vina_split

print("> Alias '%Vina' created")
print("> Alias '%Vina_split' created")

> Alias '%Vina' created
> Alias '%Vina_split' created


In [3]:
# @title **Create folders**
# @markdown This create folders for protein, ligand, experimental results and docking.

# Define path of folder
dir = os.path.abspath(".")
protein_folder = os.path.join(dir,"protein")
ligand_folder = os.path.join(dir,"ligand")
experimental_folder = os.path.join(dir,"experimental")
docking_folder = os.path.join(dir,"docking")

# Create folder if folder have not exists
folder = [protein_folder,ligand_folder,experimental_folder,docking_folder]
for f in folder:
  if os.path.exists(f):
    print("> %s already exists" % f)
  if not os.path.exists(f):
    os.mkdir(f)
    print("> %s was successfully created" % f)

> /content/protein was successfully created
> /content/ligand was successfully created
> /content/experimental was successfully created
> /content/docking was successfully created


---
---
# **Preparing the Protein**

The first step in docking is to have a structure of a given targer protein. While in some cases a high-quality comparative model will be used, most cases start with an experimentally (X-ray, NMR, cryoEM) solved three-dimensional structure. 

In such cases, a given target protein structure can be downloaded from the [Protein Data Bank (PDB)](https://www.rcsb.org/pdb) using a given accession ID.  We can directly download this structure in `.pdb` file format.

In [4]:
# @title **Generate protein PDB file**
# @markdown Enter PDB accession ID to download targeted protein.

# Define variables
PDB_ID = "7KNX" # @param {type:"string"}
protein = PDB_ID
protein_pdb = protein + ".pdb"
protein_prot = protein + "_prot"
protein_prot_pdb = protein_prot + ".pdb"

protein_pdb_file = os.path.join(protein_folder,protein_pdb)
protein_prot_file = os.path.join(protein_folder,protein_prot)
protein_prot_pdb_file = os.path.join(protein_folder,protein_prot_pdb)

# Download the protein pdb file
urllib.request.urlretrieve("http://files.rcsb.org/download/" + protein_pdb, protein_pdb_file)

print("> " + protein + ".pdb" + " successfully created in " + protein_folder)

> 7KNX.pdb successfully created in /content/protein


In [5]:
# @title **Extract protein**
# @markdown This extract **amino acids** from protein pdb file downloaded. In term of file structure, the **ATOM** lines will be extracted into a new file. Protein subunits will also be extracted as their own separate file.

# Write lines that contain #string "ATOM" into a new file
with open(protein_prot_pdb_file,"w") as g:
  f = open(protein_pdb_file,"r")
  for line in f:
    row = line.split()
    if row[0] == "ATOM":
      g.write(line)
  print("> Protein extracted")
print("> " + protein_prot_pdb + " successfully created in " + protein_folder)

# Separate protein subunits
def separate_protein(input):
  parser = PDBParser()
  io = PDBIO()
  structure  = parser.get_structure("X", input)
  chainList = [chain for chain in structure.get_chains()]
  chainLen = len(chainList)
  print("> %s of subunit(s) detected" % chainLen)
  if chainLen > 1:
    for chain in chainList:
      io.set_structure(chain)
      io.save(protein_prot_file + "_" + chain.get_id() + ".pdb")
      print("> " + protein_prot + "_%s.pdb successfuly created in " % chain.get_id() + protein_folder)

separate_protein(protein_prot_pdb_file)

> Protein extracted
> 7KNX_prot.pdb successfully created in /content/protein
> 4 of subunit(s) detected
> 7KNX_prot_A.pdb successfuly created in /content/protein
> 7KNX_prot_B.pdb successfuly created in /content/protein
> 7KNX_prot_C.pdb successfuly created in /content/protein
> 7KNX_prot_D.pdb successfuly created in /content/protein


In [110]:
# @title **Setup 3D structure viewer**
# @markdown This create 3D viewer for the tertiary protein structure.

def chain_check(input):
  chainId = []
  parser = PDBParser()
  structure  = parser.get_structure("X",input)
  numberOfChain = len(list(structure.get_chains()))
  return numberOfChain

def add_bs(object,add):
    object.addStyle({"stick":{"colorscheme":"whiteCarbon"}}) if add else None 

def add_line(object,add):
  if add:
    object.addStyle({"line":{"style":{"radius":0.1}}}) if add else None

def show_subunit(object,input,show,cols):
  j = chain_check(input) if show else 0
  if j > 1:
    for n,m in zip(range(j),cols):
      object.setStyle({"chain":(chr(65+n))},{style:{types:(m if style == "cartoon" else m + "Carbon"),"arrows":True}})
      object.addLabel("Subunit " + chr(65+n),{"fontSize":12,"fontColor":m,"backgroundOpacity": 0.7,"alignment":"topLeft"},{"chain":chr(65+n)})

def show_res(object,res,highlight):
  object.addStyle({"resi": res},{style:{types:(highlight if style == "cartoon" else highlight + "Carbon"),"arrows'":True}}) if res != "" else None
  object.addResLabels({"resi": res},{"fontSize": 12,"backgroundColor":highlight, "backgroundOpacity":0.7,"inFront":False}) if res != "" else None

def show_all_res(object,show):
  object.addResLabels({"resi": "1-9999"},{"fontSize": 12,"backgroundOpacity": 0.7,"inFront":False}) if show else None

def show_vdw_surface(object,show):
  if show:
    object.addSurface(py3Dmol.VDW,{"opacity":0.7})

# Setup 3D viewer
def view_prot(object,
              input, 
              style = "cartoon",
              types = "color",
              color = "spectrum",
              highlight = "red",
              addBS = False,
              addLine = False,
              showSubunit = False,
              showRes = "",
              showAllRes = False,
              showVDWsurface = False,
              cols = colors,
              size = (300,300)):
  print("> " + "Showing " + input[17:] + " ...")
  print("\t")
  mol1 = open(input,"r").read()
  object.addModel(mol1,"pdb")
  object.setStyle({style:{types:color,"style":"rectangle","arrows":True}})
  add_bs(object,addBS)
  add_line(object,addLine)
  show_subunit(object,input,showSubunit,cols)
  show_res(object,showRes,highlight)
  show_all_res(object,showAllRes)
  show_vdw_surface(object,showVDWsurface)
  object.setBackgroundColor("0x383838")
  object.zoomTo()
  object.show()

print("> 3D structure viewer created")

> 3D structure viewer created


In [16]:
# @title **Display 3D protein** { run: "auto" }
# @markdown Enter the protein to be viewed.

# Define variable
View = "7KNX_prot_A.pdb" #@param {type:"string"}
Model = "Ribbon Model" #@param ["Line Model","Stick Model", "Ribbon Model"]
Model_colour = "white" #@param ["red","orange","yellow","green","blue","violet","purple","white","spectrum"]
View_residues = "255,281" #@param {type:"string"}
Residues_colour = "red" #@param ["red","orange","yellow","green","blue","violet","purple","white"]
res = View_residues.split(",")
Show_all_residues = False #@param {type:"boolean"}
Show_all_subunits = True #@param {type:"boolean"}
Show_VDW_surface = False #@param {type:"boolean"}
Add_stick_model = False #@param {type:"boolean"}
Add_line_model = False #@param {type:"boolean"}
style = ""
types = ""
color = ""

# Define model
if Model == "Line Model":
  style = "line"
  types = "colorscheme"
if Model == "Stick Model":
  style = "stick"
  types = "colorscheme"
if Model == "Ribbon Model":
  style = "cartoon"
  types = "color"
  
# Display 3D structure
@interact
def viewer_one():
  try:
    mview = py3Dmol.view()
    view_prot(mview,
              input =  os.path.join(protein_folder,View), 
              style = style,
              types = types,
              color = Model_colour,
              highlight = Residues_colour,
              addBS = Add_stick_model,
              addLine = Add_line_model,
              showSubunit = Show_all_subunits,
              showRes = res,
              showAllRes = Show_all_residues,
              showVDWsurface = Show_VDW_surface)
  except Exception as e:
    print(e)

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [17]:
# @title **Parameterise protein with Gasteiger charges** 
# @markdown Enter the protein to be parameterised. This add polar hydrogen to the protein and parameterise it with Gasteiger charge using MGLtools. The protein will be converted to **pdbqt** file. In addition, this also fixes the OpenBabel out-of-format conversion bug.

# Define variables
Target_protein = "7KNX_prot_A.pdb" #@param {type:"string"}
PROTEIN = Target_protein[:-4]
PROTEIN_pdb = PROTEIN + ".pdb"
PROTEIN_pdbqt = PROTEIN + ".pdbqt"
PROTEIN_pre_pdbqt = PROTEIN + "_pre.pdbqt"

PROTEIN_pdb_pfile = os.path.join(protein_folder,PROTEIN_pdb)
PROTEIN_pdb_dfile = os.path.join(docking_folder,PROTEIN_pdb)
PROTEIN_pre_pdbqt_file = os.path.join(docking_folder,PROTEIN_pre_pdbqt)
PROTEIN_pdbqt_file = os.path.join(docking_folder,PROTEIN_pdbqt)

# Add polar hydrogen and parameterise with MGLtools
!prepare_receptor4.py -r $PROTEIN_pdb_pfile -o $PROTEIN_pre_pdbqt_file -A hydrogens -U nphs_lps -v > /dev/null 2>&1
shutil.copy(PROTEIN_pdb_pfile,docking_folder)


# Fix the OpenBabel parsing bug
with open(PROTEIN_pdbqt_file, "w") as g:
  count = 0
  f = open(PROTEIN_pre_pdbqt_file,"r")
  for line in f:
    if len(list(line)) == 81:
      oldList = list(line)
      del oldList[27]
      line = "".join(map(str, oldList))
      count += 1
    g.write(line)
os.remove(PROTEIN_pre_pdbqt_file)

print("> Total lines with error: %s" % count )
print("> Total lines fixed: %s" % count)
print("> " + PROTEIN_pdbqt + " successfully created in " + docking_folder)

/bin/bash: /usr/local/lib/libtinfo.so.5: no version information available (required by /bin/bash)
> Total lines with error: 0
> Total lines fixed: 0
> 7KNX_prot_A.pdbqt successfully created in /content/docking


---
---
# **Preparing the Ligand**

We now need to prepare the ligand that will be used for docking analysis. We will attempt to predict the docking pose onto the binding of target protein.

In [18]:
# @title **Generate ligand SMILES file**
# @markdown Enter the **canonical** SMILES of ligand.

# Define variables
Ligand_name = "C26A6" #@param {type:"string"}
SMILES = "CC1=NN2C=CC=C(C2=N1)NS(=O)(=O)C3=C(C=CC(=C3)Cl)OC" #@param {type:"string"}
LIGAND = Ligand_name
LIGAND_smi = LIGAND + ".smi"
LIGAND_mol2 = LIGAND + ".mol2"
LIGAND_pdb = LIGAND + ".pdb"
LIGAND_pdbqt = LIGAND + ".pdbqt"
LIGAND_dock = LIGAND + "_dock"

LIGAND_smi_file = os.path.join(ligand_folder,LIGAND_smi)
LIGAND_mol2_file = os.path.join(ligand_folder,LIGAND_mol2)
LIGAND_pdb_file = os.path.join(ligand_folder,LIGAND_pdb)
LIGAND_pdbqt_file = os.path.join(docking_folder,LIGAND_pdbqt)
LIGAND_dock_file = os.path.join(docking_folder,LIGAND_dock)

# Generate files
with open(LIGAND_smi_file, 'w') as g:
  g.write(SMILES)

print("> " + LIGAND_smi + " successfully created in " + ligand_folder)

> C26A6.smi successfully created in /content/ligand


In [19]:
# @title **Setup 3D structure viewer**
# @markdown This create 3D viewer for ligand with 100 000 iterations of energy minimisation via MMFF.

# Convert SMILES to rdkit.Mol with 3D coordinates
def smi_to_conf(smiles): 
  mol = Chem.MolFromSmiles(smiles)
  if mol is not None:
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    AllChem.MMFFOptimizeMolecule(mol, maxIters=100000)
    return mol
  else:
    return None

def show_atom(object,add):
  object.addPropertyLabels("atom",{},{"fontSize":12,"backgroundOpacity":0.7,"inFront":True}) if add else None

# View 3D structure
def view_lig(object,mol,showAtom=False,size=(300, 300)): 
  mblock = Chem.MolToMolBlock(mol)
  object.addModel(mblock, 'mol')
  object.setStyle({"stick":{"style":{"color":"whiteCarbon"}}})
  show_atom(object,showAtom)
  object.setBackgroundColor('0x383838')
  object.zoomTo()
  object.show()

print("> 3D structure viewer created")

> 3D structure viewer created


In [22]:
# @title **Display 3D ligand** {run: 'auto'}
# @markdown This display molecular dynamic-optimised ligand in 3D space.

# Define variables
Show_atoms = False #@param {type:"boolean"}

@interact
def viewer_two():
  mview = py3Dmol.view()
  try:
    print("> " + "Showing " + LIGAND + " ...")
    print("\t")
    conf = smi_to_conf(SMILES)
    view_lig(mview,mol = conf,showAtom = Show_atoms)
  except Exception as e:
    print(e)

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [24]:
# @title **Parameterize ligand with Gasteiger charges**
# @markdown This convert **SMILES** to **mol2** file while simultaneously performing energy minimization using the Generalized Amber Force Field (GAFF) at 100 000 iterations. After that, this add polar hydrogen to the protein and parameterise it with Gasteiger charge using MGLtools. The ligand will be converted to **pdbqt** file.

# Convert from SMILES into a 3D mol2 and pdb format
# Perform energy minimization using the GAFF(Amber)
!obabel $LIGAND_smi_file -O $LIGAND_mol2 $LIGAND_pdb --gen3d --best --canonical --minimize --ff GAFF --steps 100000 --sd > /dev/null 2>&1

# Add polar hydrogen and parameterise with MGLtools
# Add -z leads to a rigid ligand without any torsions
!prepare_ligand4.py -l $LIGAND_mol2 -o $LIGAND_pdbqt_file -U nphs_lps -v > /dev/null 2>&1

shutil.move(LIGAND_mol2, ligand_folder)
print("> " + LIGAND_smi + " successfully converted to " + LIGAND_mol2)
print("> " + LIGAND_mol2 + " successfully converted to " + LIGAND_pdbqt + " in " + docking_folder)

/bin/bash: /usr/local/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /usr/local/lib/libtinfo.so.5: no version information available (required by /bin/bash)
> C26A6.smi successfully converted to C26A6.mol2
> C26A6.mol2 successfully converted to C26A6.pdbqt in /content/docking


---
---
# **Setting Up Molecular Docking**
It is necessary to define search space for docking on a target protein through the use of grid box. The grid box is usually centered within the binding, active or allosteric site of target protein and its size should be sufficiently enough such that important binding residues are contained inside the box.

In [25]:
# @title **Setup gridbox** 
# @markdown Click once to create gridbox with 3D protein structure.

# Define the grid box
def define_grid(object,bxi,byi,bzi,bxf,byf,bzf):
  object.addBox({
      'center':{'x':bxi,'y':byi,'z':bzi},
      'dimensions': {'w':bxf,'h':byf,'d':bzf},
      'color':'blue',
      'opacity': 0.6
  })

# Generate 3D protein docking viewer
def view_dock_grid(object,input,resids,bxi,byi,bzi,bxf=10,byf=10,bzf=10, size=(300,300)):
  print("> " + "Showing " + input[17:] + " ...")
  print("\t")
  mol1 = open(input, 'r').read()
  object.addModel(mol1,'pdb')
  define_grid(object,bxi,byi,bzi,bxf,byf,bzf)
  object.setStyle({'cartoon': {'color':'spectrum'}})
  object.addStyle({'resi':resids},{'stick': {'colorscheme':'whiteCarbon'}})
  object.addResLabels({'resi':resids},{'fontSize': 12,'backgroundOpacity': 0.5,'fixed':0,'inFront':0})
  object.setBackgroundColor('0x383838')
  object.zoomTo()
  object.show()

print("> Gridbox created")

> Gridbox created


In [28]:
# @title **Place gridbox at binding site** { run: "auto" }
# @markdown Enter residues without spaces and run the cell again. Place the gridbox as near as the center of binding site 

# Define variables
Residues = "255,281" #@param {type:"string"}
X = 57 #@param {type:"slider", min:-100, max:100, step:1}
Y = 52 #@param {type:"slider", min:-100, max:100, step:1}
Z = -52 #@param {type:"slider", min:-100, max:100, step:1}

WIDTH = 16 #@param {type:"slider", min:0, max:30, step:1}
HEIGHT = 22 #@param {type:"slider", min:0, max:30, step:1}
LENGTH = 15 #@param {type:"slider", min:0, max:30, step:1}

@interact
def viewer_three():
  try:
    resids = Residues.split(",")
    mview = py3Dmol.view()
    view_dock_grid(mview,PROTEIN_pdb_dfile,resids,X,Y,Z,WIDTH,HEIGHT,LENGTH)
  except:
    return None

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [29]:
# @title **Generate docking config file**
# @markdown Click to generate config file for AutoDock Vina.

config = "config_file"

config_file = os.path.join(docking_folder,config)

with open(config_file,"w") as f:
  f.write("receptor = %s.pdbqt \n" % PROTEIN)
  f.write("ligand = %s.pdbqt \n" % LIGAND)
  f.write("\n")
  f.write("center_x = %s \n" % X)
  f.write("center_y = %s \n" % Y)
  f.write("center_z = %s \n" % Z)
  f.write("\n")
  f.write("size_x = %s \n" % WIDTH)
  f.write("size_y = %s \n" % HEIGHT)
  f.write("size_z = %s \n" % LENGTH)

print("> " + config + " successfully created in" + docking_folder)

> config_file successfully created in/content/docking


---
---
# **Performing Molecular Docking**
Autodock Vina will performing the docking stimulation with a progress bar (if running as expected). This simulation should not take longer than 5 min.

In [30]:
# @title **Run AutoDock Vina**
# @markdown Click to perform molecular docking on targeted protein.

#Changing directory to the single docking folder
os.chdir(docking_folder)

#Executing AutoDock Vina with our configuration file
%vina --config $config --out output.pdbqt --log log.txt

#Exiting the execution directory
os.chdir(dir)
print("\t")
print("> Molecular docking completed")

/bin/bash: /usr/local/lib/libtinfo.so.5: no version information available (required by /bin/bash)
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, Journal of Computational Chemistry 31 (2010)  #
# 455-461                                                       #
#                                                               #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see http://vina.scripps.edu for more information.      #
#################################################################

Detected 2 CPUs
Reading input ... done.
Set

In [31]:
# @title **Export different docking pose**
# @markdown This output 9 different docking poses as separate file from AutoDock Vina.

# Export different docking pose
!obabel -ipdbqt $docking_folder/output.pdbqt -opdb -O $LIGAND_dock_file"_.pdb" -m > /dev/null 2>&1

number = str(len([f for f in os.listdir(docking_folder) if f.startswith(LIGAND_dock)]))
print("> " + number + " molecules converted")
print("> " + "The first is " + docking_folder + LIGAND + "_dock_1.pdb")

/bin/bash: /usr/local/lib/libtinfo.so.5: no version information available (required by /bin/bash)
> 9 molecules converted
> The first is /content/dockingC26A6_dock_1.pdb


---
---
# **Preparing the Experimental Ligand**
We can compare the docking poses for verification with the experimentally solved pose for the ligand. This step is **optional**.

In [34]:
# @title **Generate experimental ligand PDB file**
# @markdown Enter the **name** assigned for the experimental ligand. T
Experimental_ligand_name = "QOV" #@param {type:"string"}
eln = Experimental_ligand_name
eln_pdb = eln + ".pdb"

eln_file = os.path.join(experimental_folder,eln)
eln_pdb_file = os.path.join(experimental_folder,eln_pdb)

# Extract exp_ligand
with open(eln_pdb_file,"w") as g:
  f = open(protein_pdb_file,'r')
  for line in f:
    row = line.split()
    if eln in row:
      g.write(line)
  print("> Experimental ligands extracted")
print("> " + eln_pdb + " successfully created in " + experimental_folder)

> Experimental ligands extracted
> QOV.pdb successfully created in /content/experimental


In [35]:
# @title **Extract experimental ligands**
# @markdown This extract all the **experimental ligands** from the pdb file and export it as separate files if any.

# Separate different experimental ligands
def separate_exp(input):
  parser = PDBParser()
  io = PDBIO()
  structure  = parser.get_structure("X", input)
  chainList = [chain for chain in structure.get_chains()]
  chainLen = len(chainList)
  print("> %s of ligands(s) detected" % chainLen)
  if chainLen > 1:
    for chain in chainList:
      io.set_structure(chain)
      io.save(eln_file + "_" + chain.get_id() + ".pdb")
      print("> " + eln + "_%s.pdb successfuly created in " % chain.get_id() + experimental_folder)

separate_exp(eln_pdb_file)

> 4 of ligands(s) detected
> QOV_A.pdb successfuly created in /content/experimental
> QOV_B.pdb successfuly created in /content/experimental
> QOV_C.pdb successfuly created in /content/experimental
> QOV_D.pdb successfuly created in /content/experimental


In [36]:
# @title **Choose exprimental model**
# @markdown Enter a target experimental ligand for comparison.
Target_exp = "QOV_A.pdb" #@param {type:"string"}
EXP = Target_exp[:-4]
EXP_pdb = EXP + ".pdb"

EXP_pdb_efile = os.path.join(experimental_folder,EXP_pdb)
EXP_pdb_dfile = os.path.join(docking_folder,EXP_pdb)

# Copy file to docking_folder
shutil.copy(EXP_pdb_efile,docking_folder)
print("> " + EXP_pdb + " successfully created in " + docking_folder)

> QOV_A.pdb successfully created in /content/docking


# **Viewing Docking Results**

The final step is to setup a 3D viewer to display and analyse our docking results.

In [114]:
# @title **Setup 3D structure viewer**
# @markdown This create 3D viewer for the protein, docked ligand and the 
# @markdown experimental ligand.

def view_dock_exp(object,input):
  if input != "":
    mol = open(input,"r").read()
    object.addModel(mol,"pdb")
    object.setStyle({"model":2},{'stick':{'colorscheme':'grayCarbon'}})
    print("> " + "Showing " + input[17:] + " ...")

def view_dock_lig(object,input,cols):
    n = input[-5:-4]
    mol = open(input,"r").read()
    object.addModel(mol,"pdb")
    object.setStyle({"model":3},{'stick':{'colorscheme':cols[int(n)-1] + 'Carbon'}})
    object.addLabel("Pose " + n,{"fontSize":12,"fontColor":cols[int(n)-1],"alignment":"topLeft"},{"model":3})
    print("> " + "Showing " + input[17:] + " (" + cols[int(n)-1] + ") ...")
 
def show_all_pose(object,input,cols,show):
  pose = sorted([f for f in os.listdir(docking_folder) if f.startswith(input[17:-5])]) if show else []
  for f,n,c in zip(pose,range(len(pose)),cols):
    fs = "".join(f)
    mol = open(os.path.join(docking_folder,fs),"r").read()
    object.addModel(mol,"pdb")
    object.setStyle({"model":n+4},{"line":{"color":c}})

def view_prot_exp_lig(object,
                      inputP, 
                      inputL = "",
                      inputE = "",
                      style = "cartoon",
                      types = "color",
                      color = "spectrum",
                      highlight = "red",
                      addBS = False,
                      addLine = False,
                      showSubunit = False,
                      showResid = "0",
                      showLabel = False,
                      showVDWsurface = False,
                      showAllPose = False,
                      cols = colors,
                      size=(1000,900)):
  print("> " + "Showing " + inputP[17:] + " ...")
  mol1 = open(inputP,"r").read()
  object.addModel(mol1,"pdb")
  object.setStyle({style:{types:color,"style":"rectangle","arrows":True}})
  add_bs(object,addBS)
  add_line(object,addLine)
  show_subunit(object,inputP,showSubunit,cols)
  show_res(object,showResid,highlight)
  show_all_res(object,showLabel)
  show_vdw_surface(object,showVDWsurface)
  view_dock_exp(object,inputE)
  view_dock_lig(object,inputL,cols)
  show_all_pose(object,inputL,cols,showAllPose)
  print("\t")
  object.setBackgroundColor("0x383838")
  object.zoomTo()
  object.show()

In [115]:
# @title **Display docking** {run: 'auto'}
# @markdown Enter the docking pose to be viewed. This display protein, docked ligand and experimental ligand in 3D space with multiple apperance choices.

# @markdown ---

# @markdown **Protein Model**
# Define variable
Model = "Line Model" #@param ["Line Model","Stick Model", "Ribbon Model"]
Model_colour = "white" #@param ["red","orange","yellow","green","blue", "violet","purple","white", "black","spectrum"]
View_residues = "" #@param {type:"string"}
Residues_colour = "blue" #@param ["red","orange","yellow","green","blue", "violet","purple","white", "black"]
res = View_residues.split(",")
Show_all_residues = False #@param {type:"boolean"}
Show_all_subunits = True #@param {type:"boolean"}
Show_VDW_surface = True #@param {type:"boolean"}
Add_stick_model = False #@param {type:"boolean"}
Add_line_model = False #@param {type:"boolean"}
style = ""
types = ""
color = ""

# @markdown ---

# @markdown **Ligand Model**
Ligand_pose = "C26A6_dock_6.pdb" #@param {type:"string"}
Show_all_pose = True #@param {type:"boolean"}

LIGAND_docked = Ligand_pose[:-4]
LIGAND_docked_pdb = LIGAND_docked + ".pdb"
LIGAND_docked_pdb_file = os.path.join(docking_folder,LIGAND_docked_pdb)

# Define model
if Model == "Line Model":
  style = "line"
  types = "colorscheme"
if Model == "Stick Model":
  style = "stick"
  types = "colorscheme"
if Model == "Ribbon Model":
  style = "cartoon"
  types = "color"
  
# Display 3D structure
@interact
def viewer_four():
  try:
    mview = py3Dmol.view(1000, 900)
    view_prot_exp_lig(mview,
                      inputP = PROTEIN_pdb_dfile, 
                      inputL = LIGAND_docked_pdb_file,
                      inputE = EXP_pdb_dfile, 
                      style = style,
                      types = types,
                      color = Model_colour,
                      highlight = Residues_colour,
                      addBS = Add_stick_model,
                      addLine = Add_line_model,
                      showSubunit = Show_all_subunits,
                      showResid = res,
                      showLabel = Show_all_residues,
                      showVDWsurface = Show_VDW_surface,
                      showAllPose = Show_all_pose)
  except Exception as e:
    print(e)

interactive(children=(Output(),), _dom_classes=('widget-interact',))